In [1]:
import json

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location":"Tokyo", "temperature":"10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location":"San Francisco","temperature":"72","unit":unit})
    if "paris" in location.lower():
        return json.dumps({"location":"Paris", "temperature":"22", "unit": unit})
    else:
        return json.dumps({"location":location, "temperature":"unknown"})

In [4]:
tools=[
    {
        "type":"function",
        "function":{
            "name":"get_current_weather",
            "description":"Get the current weather in a given location",
            "parameters": {
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"The city and state, e.g. San Francisco, CA"
                    },
                    "unit":{"type":"string", "enum":["celsius","fahrenheit"]},
                },
                "required":["location"],
            },
        },
    }
]

In [5]:
messages=[
    {"role":"user",
     "content":"What's the weather like in San Francisco, Tokyo, and Paris?"}
]

In [8]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [9]:
from openai import OpenAI 
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    messages = messages,
    tools = tools
)

In [11]:
response_message = response.choices[0].message 
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gqYiR1OhRLJmylalwnz4ofWR', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nr5I5FdwLWdxNQ7vP9eTPf6v', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_s1z7806oBJY9UeIAsB5fNfj6', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')])

ChatCompletionMessage(
	content=None, 
	role='assistant', 
	function_call=None, 
	tool_calls=[
		ChatCompletionMessageToolCall(
			id='call_X31Uc8AXen73863A7f6kMgeA', 
			function=Function(
				arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'
			), 
			type='function'
		), 
		ChatCompletionMessageToolCall(
			id='call_FaKn40r3MlTgdo3WHY8bwIZL', 
			function=Function(
				arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', 
				name='get_current_weather'
			), 
			type='function'
		), 
		ChatCompletionMessageToolCall(
			id='call_jDUnI7ocMLEouDjChOZ2gDVg', 
			function=Function(
				arguments='{"location": "Paris, France", "unit": "celsius"}', 
				name='get_current_weather'
			), 
			type='function'
		)
	]
)


In [12]:
available_functions = {
    "get_current_weather": get_current_weather
}

In [13]:
messages.append(response_message)

In [14]:
tool_calls = response_message.tool_calls

In [15]:
tool_calls

[ChatCompletionMessageToolCall(id='call_gqYiR1OhRLJmylalwnz4ofWR', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_nr5I5FdwLWdxNQ7vP9eTPf6v', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_s1z7806oBJY9UeIAsB5fNfj6', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

In [17]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args=json.loads(tool_call.function.arguments)
    function_response=function_to_call(
        location = function_args.get("location"),
        unit=function_args.get("unit")
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name":function_name,
            "content":function_response
        }
    )

In [18]:
messages

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gqYiR1OhRLJmylalwnz4ofWR', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nr5I5FdwLWdxNQ7vP9eTPf6v', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_s1z7806oBJY9UeIAsB5fNfj6', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_gqYiR1OhRLJmylalwnz4ofWR',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "celsius"}'},
 {'tool_call_id': 'call_nr5I5FdwLWdxNQ7vP9eTPf6v',
  'role': 'to

In [19]:
second_response=client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages= messages,
)

In [20]:
second_response.choices[0].message.content

"The current weather in San Francisco is 72°C and sunny, in Tokyo it's 10°C and cloudy, and in Paris it's 22°C with some sun and clouds."

"The current weather in San Francisco is 72°C and sunny, in Tokyo it's 10°C and cloudy, and in Paris it's 22°C with some sun and clouds."

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎